<center> <font size="6">Python-Spark</font></center>


1- Qual o objetivo do comando <b> cache </b> em Spark?

O comando cache instrui ao Spark da necessidade de carregar um arquivo, stream de dados ou outro tipo de informação, em memória. Após solicitar um arquivo do armazenamento ou qualquer outro tipo de processo por meio do spark, para que estes dados sejam armazenados em memória para que sejam utilizados posteriormente é necessário que se utilize o comando cache por meio da sintaxe: *.cache.

2- O mesmo código implementado em Spark é normalmente mais rápido que a implementação equivalente em MapReduce. Por quê?

O principal motivo pelo qual um código implementado em Spark é, normalmente, 100x mais rápido que a sua implementação em Hadoop com MapReduce quando em memória, ou 10x mais rápido em disco, se deve ao fato de todo o processamento em Spark ser realizado diretamente na memória principal do node Spark em que o código está rodando, não necessitando de todo o stack de trocas de informação (I/O operations, operações de troca de dados) entre o Hadoop e o hardware. Portanto, a utilização do Spark reduz muito operações em disco, que se tornam desnecessárias pelo método com que este roda os códigos.

3 - Qual a função do <b>Spark Context</b>?

O Spark context é "portão de entrada" de todas as funcionalidades do Apache Spark. Gerar o Spark Context é uma das principais tarefas esperadas de uma aplicação Spark. O Spark Context possibilita o acesso ao Cluster Spark e é ccriado após o comando SparkConf. 

4 - Explique com suas palavras o que é <b>Resilient Distributed Datasets</b> (RDD).

RDD's são a estrutura fundamental de dados do Spark, a forma como o Spark representa e armazena seus dados de forma distribuida em diversas máquinas (ou nodes), englobando, também, suas API's tornando possível acessar esses dados. Devido a descentralização dos dados é possível garantir alta taxa de persistência e disponibilidade dos dados, visto que são armazenados de forma redundante e como um "dataset" particionado por diversos servidores.

5 - <b>GroupByKey</b> é menos eficiente que <b>reduceByKey</b> em grandes datasets. Por quê?

Em maiores datasets o comando "reduceByKey" tem melhor desempenho, visto que com ele o Spark consegue combinar os resultados diferentes partições em uma mesma "chave comum", equanto o "groupByKey" somente agrupa o dataset resultando em uma combinação dos dados antes da sua separação entre os clusteres pelo RDD. O "reduceByKey" pode-ser dizer que agrupa e agrega os dados, por um indice, sem movê-los realmente, mantendo-os distribuidos o comando realiza menor movimentação dos dados economizando memória e operações.

6 - O que o código Scala abaixo faz?

![Código em Scala](image1.png)

O código descrito acima realiza as seguintes funções em descrição linha a linha:

1. Cria a variável textFile e armazena o conteudo de um arquivo texto carregado de um sistema de dados distribuidos HDFS para a mesma;

2. Cria a variável counts e faz seu "assign" para itens "mapeados" (extraidos a partir de regras) do texto carregado na variável textFile, os valores mapeados para counts serão as palavras contidas no texto, porém separando-as palavra a palavra em cada linha por meio de seus espaços.

3. Dentro do map anterior cada linha recebe outro comando map, com cada palavra sendo colocada em um arranjo (tupla) numérico, com um índice para cada palavra

4. Ainda dentro do primeiro map é feita uma redução (reduce) com os itens sendo agragados por underscores.

5. A variável counts é armazenada como um arquivo de texto após a realização dos comandos Map e Reduce

<h3> HTTP requests to NASA Kennedy Space Center WWW server

In [1]:
import re
import io
import pandas as pd

e_Pattern = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)'


dataset1 = io.open("access_log_Jul95", "r+", encoding="latin1").read()
dataset2 = io.open("access_log_Aug95", "r+", encoding="latin1").read()
dataset = dataset1+dataset2


In [2]:
dataset = dataset.split('\n')
dataset = {'raw': dataset}


In [3]:
df = pd.DataFrame(dataset)


In [4]:
df['host'] = df['raw'].str.extract('^(\S+)', expand=True)
df['timestamp'] = df['raw'].str.extract('\[([\w:/]+\s[+\-]\d{4})\]', expand=True)
df['request_Type'] = df['raw'].str.extract('"(\S+) (\S+)\s*(\S*)\s*"', expand=True)[0]
df['request_Path'] = df['raw'].str.extract('"(\S+) (\S+)\s*(\S*)\s*"', expand=True)[1]
df['request_Status'] = df['raw'].str.extract('"(\S+) (\S+)\s*(\S*)\s*"', expand=True)[2]
df['reply'] = df['raw'].str.extract('" (\d{3})', expand=True)
df['bytes'] = df['raw'].str.extract('" (\d{3}) (\S+)', expand=True)[1]
df['date'] = df['timestamp'].str.slice(0,11)



In [5]:
df = df.drop('raw', axis=1)
df = df.drop(3461612)


In [6]:
unique_hosts = len(df['host'].unique().tolist())

In [7]:
total_errors = df[df['reply'].str.contains("404")].count().reply

In [8]:
url_errors = df[df['reply'].str.contains("404")].request_Path.value_counts().head(5)

In [9]:
errors_day = df[df['reply'].str.contains("404")].date.value_counts()

In [10]:
df.bytes = pd.to_numeric(df['bytes'], errors = 'coerce')
total_bytes = df['bytes'].sum(skipna = True)/1073741824

In [11]:
print('1. Numero de hosts unicos: %d \n' % unique_hosts)
print('2. Numero total de erros 404: %d \n' % total_errors)
print('3. As 5 URLs que mais causaram erros 404: \n%s \n' % url_errors)
print('4. Quantidade de erros 404 por dia: \n%s \n' % errors_day)
print('5. Total de bytes retornados: %.4f %s' % (total_bytes, 'GB'))

1. Numero de hosts unicos: 137979 

2. Numero total de erros 404: 20901 

3. As 5 URLs que mais causaram erros 404: 
/pub/winvn/readme.txt                           2004
/pub/winvn/release.txt                          1732
/shuttle/missions/STS-69/mission-STS-69.html     683
/shuttle/missions/sts-68/ksc-upclose.gif         428
/history/apollo/a-001/a-001-patch-small.gif      384
Name: request_Path, dtype: int64 

4. Quantidade de erros 404 por dia: 
06/Jul/1995    640
19/Jul/1995    639
30/Aug/1995    571
07/Jul/1995    570
07/Aug/1995    537
13/Jul/1995    532
31/Aug/1995    526
05/Jul/1995    497
03/Jul/1995    474
12/Jul/1995    471
11/Jul/1995    471
18/Jul/1995    465
25/Jul/1995    461
20/Jul/1995    428
24/Aug/1995    420
29/Aug/1995    420
25/Aug/1995    415
14/Jul/1995    413
28/Aug/1995    410
17/Jul/1995    406
10/Jul/1995    398
08/Aug/1995    391
06/Aug/1995    373
27/Aug/1995    370
26/Aug/1995    366
04/Jul/1995    359
09/Jul/1995    348
04/Aug/1995    346
23/Aug/1995   